In [1]:
from pyspark.sql import SparkSession
# SparkSession only takes in 1 argument, it uses these functions to set attributes and then returns itself
# slow because it has to setup the JVM (and other steps)
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .getOrCreate())

# now on localhost:4040 we have a web view for Spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/21 21:50:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc = spark.sparkContext  # entry point for RDD stuff

In [3]:
nums = list(range(1_000_000))
rdd = sc.parallelize(nums)

In [4]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [5]:
# Example: map transformation
# rdd2 = rdd.map(<Function>)
# can also use anonymous lambda functions:
# rdd2 = rdd.map(lambda ARG: RETVAL)

inverses = rdd.map(lambda x: 1/x)    # ACTION: perform the function on every row in the source (rdd)

# why no error? the first row would require calculating 1/0 = undefined?

#### Div by 0 errors won't get noticed right away! Transformations don't actually force the calculation, the results only materialize after running an action (ex. collect())

In [6]:
# inverses.collect()    # ACTION: gather all the results, might use a lot of RAM so be careful

In [7]:
inverses.take(10)    # ACTION: take the first 10 results --> now we see our div by 0 error!

23/10/21 22:05:13 WARN TaskSetManager: Stage 0 contains a task of very large size (2332 KiB). The maximum recommended task size is 1000 KiB.
23/10/21 22:05:16 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.19.0.3 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 2849, in takeUpToNumLeft
    yield next(iterator)
  File "/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_26/27

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.19.0.3 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 2849, in takeUpToNumLeft
    yield next(iterator)
  File "/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*arg